In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import math

# Configuration parameters
MAP_DIR = 'maps/13semantics/'  # Output directory
DT = 0.5  # Simulated time step 
ALPHA = 0.2  # Velocity smoothing factor

# Semantic classes
SEM_DICT = [
    'bicycle_road', 
    'building', 
    'entrance', 
    'grass', 
    'obstacle', 
    'parking', 
    'pedestrian_road', 
    'tree', 
    'vehicle_road',
    'sitting_area',
    'stairs',
    'intersection_zone',
]

# Function to process a single map
def process_map(map_name, red, directory_dataset):
    """
    Generates human density, velocity, and stop maps for a given map.

    Args:
        map_name (str): Name of the map (e.g., 'stanford_bookstore0')
        red (int): Image reduction factor, should be consistent with map scale
        directory_dataset (str): Path to the video and annotation data
    """

    # Load video and annotations
    video_path = '/data/placido/Stanford_Drone_Dataset/video' + directory_dataset + 'video.mp4'
    anno_path = '/data/placido/Stanford_Drone_Dataset/annotations' + directory_dataset + 'annotations.txt'

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video stream or file")
        return

    df = pd.read_csv(anno_path, delimiter=' ', header=None)

    # Extract reference frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, 1000)  # Adjust frame position if needed
    ret, ref_frame = cap.read()
    # Create the output directory if it doesn't exist
    os.makedirs('maps/13semantics/'+map_name, exist_ok=True)
    cv2.imwrite(os.path.join(MAP_DIR, map_name, 'reference.png'), ref_frame)

    # Initialize data maps
    h, w = ref_frame.shape[:2]
    data = np.zeros((h // red, w // red))
    data_vel = np.zeros((h // red, w // red))
    data_stops = np.zeros((h // red, w // red))

    # Scale ratios for annotations
    ratioy = df[4].max() / h
    ratiox = df[3].max() / w

    # Filter for pedestrians only
    ped_df = df[df[9] == 'Pedestrian'].reset_index()

    # Process pedestrian tracks
    for trackid in ped_df[0].unique():
        data_temp = np.zeros((h // red, w // red))
        vel_temp = np.zeros((h // red, w // red))
        stop_temp = np.zeros((h // red, w // red))
        vtemp = 0

        track_df = ped_df[ped_df[0] == trackid].reset_index()
        pos = {'x': 0, 'y': 0}  # Initialize position
        vel = {'x': 0, 'y': 0}  # Initialize velocity

        for _, row in track_df.iterrows():
            center = (int((row[2] + row[4]) / 2 / ratioy / red), int((row[1] + row[3]) / 2 / ratiox / red)) 

            if center[0] < data.shape[0] and center[1] < data.shape[1] and (row[6] == 0 or row[7] == 1):
                # Update position and velocity
                prev_pos = pos.copy()  # Keep a copy of the previous position
                pos = {'x': center[0], 'y': center[1]}  # Create a new position dictionary
                vel = {
                    'x': ALPHA * (pos['x'] - prev_pos['x']) / DT + (1 - ALPHA) * vel['x'],
                    'y': ALPHA * (pos['y'] - prev_pos['y']) / DT + (1 - ALPHA) * vel['y']
                }

                # Update maps
                data_temp[center] = 1
                old_vtemp = vtemp if vtemp < 1 else old_vtemp
                vtemp = math.sqrt(vel['x']**2 + vel['y']**2)
                vtemp = vtemp if vtemp < 1 else old_vtemp # Filter out high-speed outliers
                vel_temp[center] = np.max([vel_temp[center],vtemp])
                stop_temp[center] = 1 if vtemp < 1e-20 and vtemp > 1e-100 else 0

        data = data + data_temp
        data_vel = data_vel * (vel_temp == 0) + data_vel * (vel_temp > 0)*0.5 + vel_temp * (data_vel > 0)*0.5 + vel_temp * (data_vel == 0)
        data_stops = data_stops + stop_temp        

    # Normalize and visualize density map
    data /= np.max(data)
    ref_frame_red = cv2.resize(ref_frame, (w // red, h // red), interpolation=cv2.INTER_AREA) / 510 + 0.5
    plt.figure(figsize=(6, 5))
    plt.imshow(np.multiply(ref_frame_red, np.stack((np.full(data.shape, 1), 1 - data, 1 - data), axis=2)))
    # plt.imshow(data)
    plt.axis('off')
    plt.show()

    # Save density map
    np.savetxt(os.path.join(MAP_DIR, map_name, 'humandensity-' + map_name + '-new.csv'), data * 255, delimiter=',', fmt='%3d')
    cv2.imwrite(os.path.join(MAP_DIR, map_name, 'humandensity-' + map_name + '-new.png'), data * 255)

    # Normalize and visualize velocity map
    data_vel /= np.max(data_vel)
    plt.figure(figsize=(6, 5))
    plt.imshow(np.multiply(ref_frame_red, np.stack((np.full(data_vel.shape, 1), 1 - data_vel, 1 - data_vel), axis=2)))
    # plt.imshow(data_vel)
    plt.axis('off')
    plt.show()

    # Save velocity map
    np.savetxt(os.path.join(MAP_DIR, map_name,  'humandensity-' + map_name + '-vel.csv'), data_vel, delimiter=',', fmt='%f')

    # Normalize and visualize stop map
    data_stops /= np.max(data_stops)
    plt.figure(figsize=(6, 5))
    plt.imshow(np.multiply(ref_frame_red, np.stack((np.full(data_stops.shape, 1), 1 - data_stops, 1 - data_stops), axis=2)))
    # plt.imshow(data_stops)
    plt.axis('off')
    plt.show()

    # Save stop map
    np.savetxt(os.path.join(MAP_DIR, map_name, 'humandensity-' + map_name + '-stop.csv'), data_stops, delimiter=',', fmt='%f')


if __name__ == "__main__":
    # Example usage (assuming data structure from your original code)
    locations = ['bookstore', 'bookstore', 'coupa', 'coupa', 'coupa', 'coupa', 'gates', 'gates', 'gates', 'gates', 'hyang', 'hyang', 'hyang', 'hyang', 'little', 'nexus', 'nexus', 'deathCircle']
    numbers =   [        '0',         '4',     '0',     '1',     '2',     '3',     '0',     '1',     '2',     '3',     '2',     '3',     '4',    '10',      '3',     '0',     '1',           '0']
    reds =      [        11 ,         12 ,     14 ,     14 ,     14 ,     16 ,      5 ,      6 ,      6 ,      7 ,      7 ,      7 ,      7 ,      7 ,       7 ,      5 ,      5 ,            6 ]

    for location, number, red in zip(locations, numbers, reds):
        map_name = 'stanford_' + location + number
        directory_dataset = '/' + location + '/video' + number + '/'
        process_map(map_name, red, directory_dataset)

